In [ ]:
!pip install --quiet mrjob==0.7.4


     |████████████████████████████████| 439 kB 4.7 MB/s 


In [ ]:
#mount my google drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%%file empsum.py
from mrjob.job import MRJob
import re
import uuid
import heapq as h
tempFile = "/content/gdrive/MyDrive/datasets/mr/Flist.csv"
nDigitId = 5
arr={}
def func(a):
  return a

class treeNode:
  def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = {} 
#increments the count variable with a given amount    
  def inc(self, numOccur):
        self.count += numOccur
#display tree in text. Useful for debugging        
  def disp(self, ind=1):
        print ('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

  def updateHeader(nodeToTest, targetNode):   #this version does not use recursion
    while (nodeToTest.nodeLink != None):    #Do not use recursion to traverse a linked list!
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

  def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:#check if orderedItems[0] in retTree.children
        inTree.children[items[0]].inc(count) #incrament count
    else:   #add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #update header table 
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            treeNode.updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:#call updateTree() with remaining ordered items
        treeNode.updateTree(items[1::], inTree.children[items[0]], headerTable, count)


  def createTree(dataSet, minSup=1): #create FP-tree from dataset but don't mine
    headerTable = {}
    #go over dataSet twice
    for trans in dataSet:#first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    for k in list(headerTable):  #remove items not meeting minSup
        if headerTable[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    #print 'freqItemSet: ',freqItemSet
    if len(freqItemSet) == 0: return None, None  #if no items meet min support -->get out
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link 
    #print 'headerTable: ',headerTable
    retTree = treeNode('Null Set', 1, None) #create tree
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        localD = {}
        for item in tranSet:  #put transaction items in order
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            treeNode.updateTree(orderedItems, retTree, headerTable, count)#populate tree with ordered freq itemset
    return retTree, headerTable #return tree and header table

  

def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

class EmpSum(MRJob):
    def mapper(self, key, line):
      for word in line.split(","):
        if(word != ""):
            yield word.lower(), 1

    def reducer(self, word, counts):
        yield word, sum(counts)

class pfp(MRJob):
    def mapper_init(self):
        f = open(tempFile, 'r')
        lines = f.readlines()
        arr = {}
        gid=[]
        id=[]
        for i in range(4):
            x=str(uuid.uuid4())
            id.append(i)
        j=0
        for line in lines:
          record = line.split(',')
          arr[ record[0] ] = int(record[1])
          gid.append(record[0])
          j=j+1
        #print(dno_avgs)
        self.inx=None 
        self.arr = arr
        self.gid=gid
        self.id=id

    def mapper(self, _, line):
        f = open("/content/gdrive/MyDrive/datasets/DataSetAA.csv", 'r')
        lines = f.readlines()
        self.a = []
        for i in line.split(","):
          if(i!=""):
            self.a.append(i)
        n=len(self.a)
        for j in range(n):
          self.buf=[]
          try:
            hashnum = self.id[(self.gid.index(self.a[n-j-1].lower()))%4]
            self.inx=self.gid.index(self.a[n-j-1].lower())%4
          except:
            hashnum = ""
          if hashnum != "":
            self.gid[self.inx]= None
            for i in range(n-j):
              self.buf.append(self.a[i])
          #if(len(self.buf)!=0):
          yield(hashnum,self.buf)

    def reducer(self, word, counts):
          yield ("word", counts)


if __name__ == '__main__':
    
    inputFile = "/content/gdrive/MyDrive/datasets/DataSetAA.csv"
    outFile = tempFile
    mr_job = EmpSum(args=[inputFile])
    with mr_job.make_runner() as runner:
      runner.run()
      f = open(outFile, "w") 
      for key, value in mr_job.parse_output(runner.cat_output()):
        s = f'{key},{value}'
        print(s, file = f)
      f.close()

    mr_job2 = pfp(args=[inputFile])
    with mr_job2.make_runner() as runner:
      runner.run()
      for key, value in mr_job2.parse_output(runner.cat_output()):
        print(key,value)


Writing empsum.py


In [ ]:
!python empsum.py "/content/gdrive/MyDrive/datasets/DataSetAA.csv"

No configs specified for inline runner
No configs specified for inline runner

Error while reading from /tmp/empsum.root.20221217.174014.581633/step/000/reducer/00000/input:

Traceback (most recent call last):
  File "empsum.py", line 151, in <module>
    runner.run()
  File "/usr/local/lib/python3.8/dist-packages/mrjob/runner.py", line 503, in run
    self._run()
  File "/usr/local/lib/python3.8/dist-packages/mrjob/sim.py", line 161, in _run
    self._run_step(step, step_num)
  File "/usr/local/lib/python3.8/dist-packages/mrjob/sim.py", line 170, in _run_step
    self._run_streaming_step(step, step_num)
  File "/usr/local/lib/python3.8/dist-packages/mrjob/sim.py", line 187, in _run_streaming_step
    self._run_reducers(step_num, num_reducer_tasks)
  File "/usr/local/lib/python3.8/dist-packages/mrjob/sim.py", line 289, in _run_reducers
    self._run_multiple(
  File "/usr/local/lib/python3.8/dist-packages/mrjob/sim.py", line 130, in _run_multiple
    func()
  File "/usr/local/lib/pytho

In [ ]:
class treeNode:
  def __init__(self, nameValue, numOccur, parentNode):
        self.name = nameValue
        self.count = numOccur
        self.nodeLink = None
        self.parent = parentNode      #needs to be updated
        self.children = {} 
#increments the count variable with a given amount    
  def inc(self, numOccur):
        self.count += numOccur
#display tree in text. Useful for debugging        
  def disp(self, ind=1):
        print ('  '*ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind+1)

  def updateHeader(nodeToTest, targetNode):   #this version does not use recursion
    while (nodeToTest.nodeLink != None):    #Do not use recursion to traverse a linked list!
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode

  def updateTree(items, inTree, headerTable, count):
    if items[0] in inTree.children:#check if orderedItems[0] in retTree.children
        inTree.children[items[0]].inc(count) #incrament count
    else:   #add items[0] to inTree.children
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        if headerTable[items[0]][1] == None: #update header table 
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            treeNode.updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    if len(items) > 1:#call updateTree() with remaining ordered items
        treeNode.updateTree(items[1::], inTree.children[items[0]], headerTable, count)


  def createTree(dataSet, minSup=1): #create FP-tree from dataset but don't mine
    headerTable = {}
    #go over dataSet twice
    for trans in dataSet:#first pass counts frequency of occurance
        for item in trans:
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    for k in list(headerTable):  #remove items not meeting minSup
        if headerTable[k] < minSup: 
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    #print 'freqItemSet: ',freqItemSet
    if len(freqItemSet) == 0: return None, None  #if no items meet min support -->get out
    for k in headerTable:
        headerTable[k] = [headerTable[k], None] #reformat headerTable to use Node link 
    #print 'headerTable: ',headerTable
    retTree = treeNode('Null Set', 1, None) #create tree
    for tranSet, count in dataSet.items():  #go through dataset 2nd time
        localD = {}
        for item in tranSet:  #put transaction items in order
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            treeNode.updateTree(orderedItems, retTree, headerTable, count)#populate tree with ordered freq itemset
    return retTree, headerTable #return tree and header table

  

def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict


In [ ]:
pip install mrjob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from collections import defaultdict
from itertools import chain, combinations
from operator import itemgetter

In [ ]:
class FPNode:
    def __init__(self, item, count, parent):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.link = None

    def add(self, transaction, item_freq):
        if transaction:
            item = transaction[0]
            child = self.children.get(item)
            if child is None:
                child = FPNode(item, item_freq[item], self)
                self.children[item] = child
                if self.link is not None:
                    child.link = self.link.children.get(item)
                    self.link.children[item] = child
                else:
                    child.link = None
            child.count += 1
            child.add(transaction[1:], item_freq)

    def prefix_paths(self, item):
        paths = []
        if self.item == item:
            paths.append([self])
        for child in self.children.values():
            for path in child.prefix_paths(item):
                paths.append([self] + path)
        return paths


In [ ]:
%%file temp.py
class node:

    def __init__(self, name, freq):
        self.name = name
        self.freq = freq
        self.children=[]
        self.child={}
        
    def disp(self, ind=1):
        print ('  '*ind, self.name, '  ', self.freq)
        for child in self.child.values():
            child.disp(ind+2)


class FPNode:
      def __init__(self, item, parent, count=1):
          self.item = item
          self.parent = parent
          self.count = count
          self.children = []

      def add_child(self, child):
          self.children.append(child)

      def __repr__(self):
          return f"FPNode(item={self.item}, count={self.count})"

class FPTree:
    def __init__(self, transactions, min_support=1):
        # Create a frequency map of the items in the transactions
        item_freq = defaultdict(int)
        for transaction in transactions:
            for item in transaction:
                item_freq[item] += 1

        # Filter out items that don't meet the minimum support
        items = [item for item, count in item_freq.items() if count >= min_support]

        # Sort the items by frequency in decreasing order
        items.sort(key=lambda x: item_freq[x], reverse=True)

        # Create the FP-tree
        self.root = FPNode("root", None, None)
        for transaction in transactions:
            sorted_items = [item for item in transaction if item in items]
            self._add_transaction(sorted_items, self.root, item_freq)

    def _add_transaction(self, items, node, item_freq):
        # If there are no items left, return
        if not items:
            return

        # Get the first item in the list
        item = items[0]

        # Try to find a child node for the item
        child = next((x for x in node.children if x.item == item), None)

        # If a child node doesn't exist, create one
        if not child:
            child = FPNode(item, node, item_freq[item])
            node.add_child(child)

        # Increment the count for the item
        child.count += 1

        # Add the remaining items to the FP-tree
        self._add_transaction(items[1:], child, item_freq)

    def __repr__(self):
        return self._repr(self.root, 0)

    def _repr(self, node, depth):
        result = ""
        if node.children:
            result += " " * depth + str(node) + "\n"
            for child in node.children:
                result += self._repr(child, depth + 1)
        else:
            result += " " * depth + str(node) + "\n"
        return result

In [ ]:
%%file empsum1.py
from mrjob.job import MRJob
from temp import FPNode
from temp import FPTree
from temp import node
import numpy as np
import re
import uuid
import json
import heapq as h
tempFile = "/content/gdrive/MyDrive/Flist.csv"
nDigitId = 5

    
def update(n,k):
    if k in n.children:
        n.child[k].freq+=1;
        
    else:
        n.child[k]=node(k,1)
        n.children.append(k)
       
class EmpSum(MRJob):
    def mapper(self, key, line):
      for word in line.split(","):
        if(word != ""):
            yield word.lower(), 1

    def reducer(self, word, counts):
        yield word, sum(counts)


from collections import defaultdict

class pfp(MRJob):
    

    def mapper_init(self):
        f = open(tempFile, 'r')
        lines = f.readlines()
        arr = {}
        gid=[]
        id=[]
        for i in range(4):
            x=str(uuid.uuid4())
            id.append(i)
        j=0
        for line in lines:
          record = line.split(',')
          arr[ record[0] ] = int(record[1])
          gid.append(record[0])
          j=j+1
        #print(dno_avgs)
        self.inx=None 
        self.arr = arr
        self.gid=gid
        self.id=id

    def mapper(self, _, line):
        f = open("/content/gdrive/MyDrive/DataSetAA.csv", 'r')
        lines = f.readlines()
        self.a = []
        for i in line.split(","):
          if(i!=""):
            self.a.append(i)
        n=len(self.a)
        for j in range(n):
          self.buf=[]
          try:
            hashnum = self.id[(self.gid.index(self.a[n-j-1].lower()))%4]
            self.inx=self.gid.index(self.a[n-j-1].lower())%4
          except:
            hashnum = ""
          if hashnum != "":
            self.gid[self.inx]= None
            for i in range(n-j):
              self.buf.append(self.a[i])
          if(len(self.buf)!=0):
            yield(hashnum,list(self.buf))
    def reducer_init(self):
        f = open(tempFile, 'r')
        lines = f.readlines()
        arr = {}
        gid=[]
        id=[]
        for i in range(4):
            id.append(i)
        j=0
        for line in lines:
          record = line.split(',')
          arr[ record[0] ] = int(record[1])
          gid.append(record[0])
          j=j+1
        #print(dno_avgs)
        self.arr = arr
        self.gid=gid
        self.id=id
    def reducer(self,key,value):
      ml=json.dumps(list(value))     
      # itr= self.id.index(key)
      # x=[]
      # for p in range(4):
      #   if (itr + p*4)<len(self.arr):
      #     x.append(self.gid[itr + p*4])
      # null=FPNode(ml,None)
      
      yield 0, ml



if __name__ == '__main__':
    
    inputFile = "/content/gdrive/MyDrive/DataSetAA.csv"
    outFile = tempFile
    mr_job = EmpSum(args=[inputFile])
    with mr_job.make_runner() as runner:
      runner.run()
      f = open(outFile, "w") 
      for key, value in mr_job.parse_output(runner.cat_output()):
        s = f'{key},{value}'
        print(s, file = f)
      f.close()

    mr_job2 = pfp(args=[inputFile])
    with mr_job2.make_runner() as runner:
      runner.run()
      for key, value in mr_job2.parse_output(runner.cat_output()):
        print(key,value)


Overwriting empsum1.py


In [ ]:

!python empsum.py "/content/gdrive/MyDrive/datasets/DataSetAA.csv"

No configs specified for inline runner
No configs specified for inline runner

Error while reading from /tmp/empsum.root.20221217.193311.530273/step/000/reducer/00000/input:

Traceback (most recent call last):
  File "empsum.py", line 151, in <module>
    runner.run()
  File "/usr/local/lib/python3.8/dist-packages/mrjob/runner.py", line 503, in run
    self._run()
  File "/usr/local/lib/python3.8/dist-packages/mrjob/sim.py", line 161, in _run
    self._run_step(step, step_num)
  File "/usr/local/lib/python3.8/dist-packages/mrjob/sim.py", line 170, in _run_step
    self._run_streaming_step(step, step_num)
  File "/usr/local/lib/python3.8/dist-packages/mrjob/sim.py", line 187, in _run_streaming_step
    self._run_reducers(step_num, num_reducer_tasks)
  File "/usr/local/lib/python3.8/dist-packages/mrjob/sim.py", line 289, in _run_reducers
    self._run_multiple(
  File "/usr/local/lib/python3.8/dist-packages/mrjob/sim.py", line 130, in _run_multiple
    func()
  File "/usr/local/lib/pytho